Dou et al. (2020) model

Agora em Julia versão 1.5.3

This script simulates the model

Calculando a derivada fixando os parâmetros iguais aos de Dou et al. e substituindo pelo $\beta$ encontrado na última estimação com 2 clusters

In [1]:
using LinearAlgebra, Statistics

using DataFrames
using BenchmarkTools


In [2]:
using Distributed
nprocs = 2
addprocs(nprocs);

@everywhere using NBInclude
@everywhere @nbinclude("solve_model.ipynb")

#importou a função solve_tree

pids = workers()
aux = pids[nprocs]

3

In [3]:
#Vh/D, L/D, Dj/D
data1 = [0.86, 0.19, 0.8];
n1 = 49

data2 = [2.65, 0.69, 0.74];
n2 = 17

#ρ, β, c0, λj
game_parameters = [0.639, 8.553, 0.01, 0.011];

In [4]:
A2 = @spawnat aux solve_tree(data2, game_parameters)

A1 = solve_tree(data1, game_parameters);

A2 = fetch(A2);

       From worker 3:	 38.076764 seconds (142.85 M allocations: 3.216 GiB, 1.38% gc time)
42.700561 seconds (142.85 M allocations: 3.216 GiB, 1.29% gc time)


Simulation parameters

In [5]:
#θs0, θj0, μ
simulation_parameters = [11.0, 90.0, 4.543];

### Criando funções para simular o jogo

In [6]:
#cost function
function Ct(t, c0=c0, c1=c1)
    
    #didn't use D because is normalized to D == 1.0
    #cost at period t=0(index1) is 0
    if(t <= 1)
        return 0
    else
        return c0 + c1 * (t-1) #test to make index==1 be t==0
    end
end



#liquidation payoffs
function s_L(t, L, Ds, Dj, c0, c1)
    return min(L - Ct(t, c0, c1), Ds)
end


function j_L(t, L, Ds, Dj, c0, c1)
    return min(L - Ct(t, c0, c1) - s_L(t, L, Ds, Dj, c0, c1), Dj)
end


function proposal(Pkt_array, t, hkt, lmt)
    
    #pkt array será sempre do propositor, quem responder às propostas apenas olhará o seu valor de continuação
    
    return policy, payoff_prop, payment, lm_next = Pkt_array[t, hkt, lmt, [end,end-1, 1, 2]]
    
end
    

#maximum value of reorganization each period
function Vt(Vmax, ρ, t)

    if(t <=1)
        return Vmax
    else
        #(t-2) instead of (t-1) because we shifted the indexes in the game so as to include t==0 at index==1
        return ρ^(t-2) * Vmax
    end


end

# answer_liq
function answer_liq(m_L, t, hm_next, payoff_prop, L, Ds, Dj, Vmax, ρ, c0, c1)
    
    liq = (m_L(t, L, Ds, Dj, c0, c1), Vt(Vmax, ρ, t) * hm_next/100 - Ct(t, c0, c1) - payoff_prop)
    
    payoff_liq, answer = findmax(liq)
    
    return payoff_liq, answer
end

function answer_reorg(payment, Cont_val, t, hm_next, lm_next, lk_next)
    
    reorg_value = (payment, Cont_val[t+1, hm_next, lm_next, lk_next])

    payoff_reorg, answer = findmax(reorg_value)
    
    return payoff_reorg, answer
end

function update_beliefs(propositor, hk_next, hm_next, lk_next, lm_next)
    
    if(propositor=="s")
        hst = hk_next
        lst = lk_next
        
        hjt = hm_next
        ljt = lm_next
        
    else
        hst = hm_next
        lst = lm_next
        
        hjt = hk_next
        ljt = lk_next
    end
    
    
    return hst, lst, hjt, ljt
end

        

update_beliefs (generic function with 1 method)

In [7]:
#código para tirar um draw da beta

function draw_beta(hkt, β=game_parameters[2], grid=100)

    u = rand()

    if(hkt == grid)
        
        return grid

    else
        
        x = 1.0 - exp(1.0/β * (log(1.0 - u) + β * log(1.0 - hkt/grid)))
        x = round(x * 100, digits=0)
#         return Int64(x * 100) #to convert in an integer
        return Int64(x)
        
    end
end

        

# @code_warntype draw_beta(10)



draw_beta (generic function with 3 methods)

In [8]:
function choose_parameters(propositor, hst, lst, hjt, ljt, Pst_array, Pjt_array, s_W, j_W)
    
    if(propositor=="s")
        
        Pkt_array = Pst_array
        Cont_val = j_W
        prop_index = 1
        respondent_index = 2 #índice de j, para organizar o payoff
        m_L = j_L

        hkt = hst
        lkt = lst
        
        hmt = hjt
        lmt = ljt


        lk_next = hkt

        hk_next = draw_beta(hkt)
        hm_next = draw_beta(hmt)
        
    elseif(propositor=="j")
        Pkt_array = Pjt_array
        Cont_val = s_W
        prop_index = 2
        respondent_index = 1
        m_L = s_L

        hkt = hjt
        lkt = ljt
        
        hmt = hst
        lmt = lst


        lk_next = hkt


        hk_next = draw_beta(hkt)
        hm_next = draw_beta(hmt)

    else
        
        println("error: propositor not valid")
        
    end
    
    return Pkt_array, Cont_val, prop_index, respondent_index, m_L, hkt, lkt, hmt, lmt, lk_next, hk_next, hm_next
end




# #test
# Hs0 = 30
# Hj0 = 28


# hst = Hs0
# hjt = Hj0

# #assumindo que lower bounds nos períodos iniciais são as próprias habilidades iniciais
# lst = hst
# ljt = hjt




# choose_parameters("s", hst, lst, hjt, ljt);
    
    

In [9]:
function simulate_game(data, game_parameters, simulation_parameters, s_W=s_W, j_W=j_W, Pst_array=Pst_array, Pjt_array=Pjt_array, t=1, grid=100)
    
    
    Vmax = data[1]
    L = data[2]
    Dj = data[3]
    Ds = one(1.0) - Dj
    
    
    c0 = game_parameters[3];
    c1 = c0/30;

    ρ = game_parameters[1];
    λj = game_parameters[4];

    
    
#     hst = Hs0
#     hjt = Hj0
    
    hst = Int64(simulation_parameters[1])
    hjt = Int64(simulation_parameters[2])

    #assumindo que lower bounds nos períodos iniciais são as próprias habilidades iniciais
    lst = hst
    ljt = hjt
    
    result = zeros(5);
    
    #number of observed proposals
    observed_proposals = 0.0
    
    
    #recovering "T+1" from s_W 
    T = size(s_W,1)
    
    
    while(result[1]==0.0 && t < T)
        
        
        u = rand()

        if(u < λj)
            propositor = "j"

        else
            propositor = "s"

        end

        
        #setting the default variables according to the propositor
        Pkt_array, Cont_val, prop_index, respondent_index, m_L, hkt, lkt, hmt, lmt, lk_next, hk_next, hm_next = choose_parameters(propositor, hst, lst, hjt, ljt, Pst_array, Pjt_array, s_W, j_W)
        
        #proposal ####
        policy, payoff_prop, payment, lm_next = proposal(Pkt_array, t, hkt, lmt)

        #lm_next is the update of the adversary's lower bound
        if(lm_next>=grid+1 || policy!= 3.0) #para evitar update de lowerbound quando proponente não propõe reorg
            lm_next = lmt
        else
            lm_next = Int64(lm_next)
        end
        
        
        
        if(policy==3.0)
            
            observed_proposals += 1.0
    
            payoff_respondent, answer = answer_reorg(payment, Cont_val, t, hm_next, lm_next, lk_next)


            if(answer==1.0)

                result[prop_index] = payoff_prop
                result[respondent_index] = payoff_respondent
                result[3] = 3.0
                result[4] = t
                result[5] = observed_proposals

            else
                
                t+=1

                hst, lst, hjt, ljt = update_beliefs(propositor, hk_next, hm_next, lk_next, lm_next)
            end

        elseif(policy==2.0)
            
            t+=1
            
            hst, lst, hjt, ljt = update_beliefs(propositor, hk_next, hm_next, lk_next, lm_next)
            
            

        else      
            #(policy==1.0)
            
            observed_proposals += 1.0

            payoff_respondent, answer = answer_liq(m_L, t, hm_next, payoff_prop, L, Ds, Dj, Vmax, ρ, c0, c1)
            
            

            if(answer==1.0)

                result[prop_index] = payoff_prop
                result[respondent_index] = payoff_respondent
                result[3] = 1.0
                result[4] = t
                result[5] = observed_proposals

            else

                result[prop_index] = payoff_prop
                result[respondent_index] = payoff_respondent
                result[3] = 3.0
                result[4] = t
                result[5] = observed_proposals


            end
        end


        if(t==T)
            
            result[1] = s_W[T, 1, 1, 1]#todos os valores finais de S são iguais, então acessarei o índice 1
            result[2] = j_W[T, 1, 1, 1]
            result[3] = 1.0
            result[4] = t
            result[5] = max(observed_proposals, 1.0)
            
        end
        




    end
    
    
    return result
end

        
        
#precisa colocar uma condição IF aqui para caso o jogo vá até o último período?
    

simulate_game (generic function with 7 methods)

In [10]:
#unit testing
@btime simulate_game(data1, game_parameters, simulation_parameters, A1[1], A1[2], A1[3], A1[4])

  3.971 μs (31 allocations: 1.27 KiB)


5-element Array{Float64,1}:
 0.179
 0.13657513579999997
 3.0
 4.0
 1.0

In [11]:
function loop_simulations(S, N, game_parameters, simulation_parameters, data1, data2, A1, A2, n1)
    
    
    
    
    
    #S is the number of simulations
    #N is the number of observations
    #data1 is the center of the first cluster
    #A1 is the collection of arrays from the first cluster (s_W, j_W, Pst, Pjt)
    #n1 is the number of observations in the first cluster
    
    
    
    
    #no futuro os argumentos podem ser arrays de arrays, assim ele faz o loop para cada cluster
    Results = zeros(S, N, 7)

    #últimas entradas de Results são os valores das dívidas
    
    
    for s in 1:S
        for n in 1:N
            if(n <= n1)
                Results[s,n,1:5] .= simulate_game(data1, game_parameters, simulation_parameters, A1[1], A1[2], A1[3], A1[4])
                Results[s,n, end-1] = A1[5]
                Results[s,n, end] = A1[6]
            else
                Results[s,n,1:5] .= simulate_game(data2, game_parameters, simulation_parameters, A2[1], A2[2], A2[3], A2[4])
                Results[s,n, end-1] = A2[5]
                Results[s,n, end] = A2[6]
            end
            
        end
    end

    return Results
    
end

    
# loop_simulations(40, 75, θs0, θj0, μ, λj);

In [12]:
#unit testing
S = 40
N = 66

using Random

Random.seed!(2021)

Results = loop_simulations(S, N, game_parameters, simulation_parameters, data1, data2, A1, A2, n1)

40×66×7 Array{Float64,3}:
[:, :, 1] =
 0.179333  0.179333  0.179333  0.181488  …  0.261496  0.26      0.26
 0.178667  0.179333  0.275091  0.179333     0.309035  0.379928  0.26
 0.179333  0.179333  0.179333  0.179        0.403541  0.33271   0.403541
 0.179333  0.179     0.179333  0.179333     0.26      0.949208  0.451406
 0.179333  0.179333  0.179     0.179333     0.427474  0.26      0.26
 0.179     0.179333  0.179333  0.179333  …  0.26      0.522452  0.26
 0.179333  0.179     0.179     0.185764     0.33271   0.285562  0.379928
 0.179333  0.179     0.179333  0.179        0.33271   0.289729  0.309035
 0.179     0.179     0.179333  0.40151      0.427474  0.26      0.759433
 0.179     0.312877  0.179333  0.226129     0.26      0.26      0.261496
 0.347866  0.179     0.179     0.199451  …  0.475041  0.569878  0.26
 0.179333  0.179333  0.179333  0.200586     0.475041  0.26      0.26
 0.179333  0.179     0.179     0.179333     0.33271   0.26      0.285562
 ⋮                                   

In [13]:
Results[1,:, 2]

66-element Array{Float64,1}:
 0.3100814
 0.31557680000000005
 0.31557680000000005
 0.02230333655176977
 0.32107220000000003
 0.32107220000000003
 0.143598257
 0.332063
 0.3100814
 0.31557680000000005
 0.304586
 0.143598257
 0.332063
 ⋮
 0.4183333333333332
 0.43260388888888884
 1.8688382320579078
 0.4328248014681348
 0.4324810460460461
 0.4183333333333332
 0.4324776192859527
 0.4183333333333332
 0.4326642909576242
 1.2739563585730553
 0.41866666666666663
 0.4183333333333332

In [14]:
s=1


x = Results[s,:, 2] ./ (Results[s, :, 1] .+ Results[s,:, 2])

66-element Array{Float64,1}:
 0.6335759405690143
 0.6376446525241216
 0.6376446525241216
 0.10944195968778397
 0.6416240001609361
 0.6416240001609361
 0.4451302940548746
 0.6493261260509624
 0.6335759405690143
 0.6376446525241216
 0.6294148198253428
 0.4451302940548746
 0.6493261260509624
 ⋮
 0.6167076167076166
 0.5029825009786767
 0.8199239624901448
 0.5175064204013511
 0.4765515094967668
 0.6167076167076166
 0.5651913531339122
 0.6167076167076166
 0.40300064049036055
 0.8296945379390959
 0.6168958742632613
 0.6167076167076166

In [15]:
using StatsBase
summarystats(x)

Summary Stats:
Length:         66
Missing Count:  0
Mean:           0.561257
Minimum:        0.078153
1st Quartile:   0.445130
Median:         0.633576
3rd Quartile:   0.641624
Maximum:        0.829874


In [16]:
searchsorted(x, Inf)

67:66

In [17]:
findmax(x)

(0.8298742562240793, 53)

In [18]:
x[26]

0.6376446525241216

In [19]:
Results[s, 26, :]

7-element Array{Float64,1}:
 0.17933333333333334
 0.31557680000000005
 3.0
 3.0
 1.0
 0.19999999999999996
 0.8

In [20]:
x[x .== Inf]

Float64[]

In [21]:
function simulate_moments(S, N, game_parameters, simulation_parameters, data1, data2, A1, A2, n1)
    
    
    Results = loop_simulations(S, N, game_parameters, simulation_parameters, data1, data2, A1, A2, n1)
    
    Moments = zeros(S, 8)
    
    μ = simulation_parameters[3]
    for s in 1:S

            DF = DataFrame()
            DF.payoff_s = Results[s,:,1];
            DF.payoff_j = Results[s,:,2];
            DF.out = Results[s,:,3];
            DF.t = Results[s,:,4];
            DF.observed_proposals = Results[s,:,5];
            DF.Ds = Results[s,:,6];
            DF.Dj = Results[s,:,7];


            DF.outcome = ifelse.(DF.out .==3.0, "R", "L");


            #desfazendo o deslocamento no índice de t
            DF.t = DF.t .- 1.0;

            # DF.incourt = ifelse.(DF.t .> 0.0, "incourt", "precourt");

            INCOURT = filter(DF -> DF.t .> 0.0, DF);
            PRECOURT = filter(DF -> DF.t .<= 0.0, DF);

            PRECOURT_REORG = filter(PRECOURT -> PRECOURT.outcome .== "R", PRECOURT)
            INCOURT_REORG = filter(INCOURT -> INCOURT.outcome .== "R", INCOURT)

            #cálculo dos momentos
            #1. avg log number of months between observed proposals incourt

            mm1 = INCOURT
            mm1 = log.(mm1.t .* μ ./ mm1.observed_proposals)
            mm1 = mean(mm1)

            #2. fraction reorganized given that the case went into court

            mm2 = size(INCOURT_REORG,1)/ size(INCOURT,1)

            #3. ln duration of court cases in months

            mm3 = INCOURT
            mm3 = filter(mm3 -> mm3.t .> 0.0, mm3) #removendo os casos 0 para não poluir a média
            mm3.t = mm3.t .* μ
            mm3 = mean(log.(mm3.t)) #log here uses exp as base, so it's the same as ln

            #4. fraction of cases incourt
            mm4 = size(INCOURT,1) / size(DF, 1)

            #5. avg recovery rate for senior given precourt REORGANIZATION

            #aqui nós temos Ds, então é mais fácil. Preciso automatizar isso no código quando for tudo escalado

            mm5 = PRECOURT_REORG
            mm5.R_s = mm5.payoff_s ./ mm5.Ds
            mm5 = mean(mm5.R_s)



            #6. avg recovery rate for junior given precourt REORGANIZATION

            #aqui nós temos Ds, então é mais fácil. Preciso automatizar isso no código quando for tudo escalado

            mm6 = PRECOURT_REORG
            mm6.R_j = mm6.payoff_j ./ mm6.Dj
            mm6 = mean(mm6.R_j)


            #7. junior avg fraction gain given incourt REORGANIZATION

            mm7 = INCOURT_REORG
            mm7 = mean(mm7.payoff_j ./ (mm7.payoff_j .+ mm7.payoff_s )) 

            #8. total recovery rate given incourt REORGANIZATION

            mm8 = INCOURT_REORG
            mm8 = mean( mm8.payoff_s .+ mm8.payoff_j)
        
        
            Moments[s,:] .= [mm1, mm2, mm3, mm4, mm5, mm6, mm7, mm8]

            
            #if there are no INCOURT or no PRECOURT cases, it will return NaN
            #thus, we replace it by zero
            Moments[s,:] .= replace!(Moments[s,:], NaN=> zero(0.0))

        end
    return vec(mean(Moments, dims=1))
end



  

simulate_moments (generic function with 1 method)

In [23]:
using Random

Random.seed!(2021)

#test
S = 100
N = 66

simulated_moments = simulate_moments(S, N, game_parameters, simulation_parameters, data1, data2, A1, A2, n1)

8-element Array{Float64,1}:
 2.220022962758409
 0.8853575348262847
 2.2618368968261358
 0.9896969696969697
 0.6054418352968354
 0.5345749164581954
 0.5291769118653423
 0.5100150830412142

Calculando a matriz de pesos ótima com os resultados dessa estimação


Two-step variance covariance estimator of W
https://notes.quantecon.org/submission/5b3db2ceb9eab00015b89f93



In [24]:
#data_moments
data_moments = [2.11, 0.843, 2.76, 0.969, 0.6235, 0.5569, 0.773, 0.497];

In [25]:
function omega(N, data_moments, simulated_moments)
    
    error = (simulated_moments .- data_moments) ./ data_moments
    
    omega = 1/N * error * transpose(error)
    
    return omega
end

omega (generic function with 1 method)

In [26]:
W_2step = pinv(omega(N, data_moments, simulated_moments))

8×8 Array{Float64,2}:
   9.0388     8.70989  -31.2876    3.70248  …  -6.94905  -54.6771    4.53942
   8.70989    8.39296  -30.1491    3.56775     -6.69619  -52.6875    4.37424
 -31.2876   -30.1491   108.301   -12.8161      24.054    189.263   -15.7131
   3.70248    3.56775  -12.8161    1.51661     -2.84648  -22.3969    1.85944
  -5.0205    -4.83781   17.3783   -2.0565       3.85977   30.3698   -2.52137
  -6.94905   -6.69619   24.054    -2.84648  …   5.34245   42.0359   -3.48992
 -54.6771   -52.6875   189.263   -22.3969      42.0359   330.75    -27.4596
   4.53942    4.37424  -15.7131    1.85944     -3.48992  -27.4596    2.27976

In [27]:
#pesos dados a cada um dos momentos
diag(W_2step)

8-element Array{Float64,1}:
   9.038796990009615
   8.39295849540429
 108.3013383421914
   1.5166132469418885
   2.7885804316048124
   5.342451905470017
 330.7497607555904
   2.279764469905542

Calculando a derivada dos momentos em relação aos parâmetros

In [28]:
function simulate_new_moments(i, S, N, game_parameters, simulation_parameters, data1, data2, A1, A2, n1)
    
    
    println("parameter ",i, game_parameters, simulation_parameters)
        
    θs0, θj0, μ = simulation_parameters;
    θs0 = Int64(round(θs0, digits=0));
    θj0 = Int64(round(θj0, digits=0));

    λj = game_parameters[4];
    new_simulated_moments = simulate_moments(S, N, game_parameters, simulation_parameters, data1, data2, A1, A2, n1) 
    
    return new_simulated_moments
end

simulate_new_moments (generic function with 1 method)

In [29]:
 
function derivative_moments(S, N, game_parameters, simulation_parameters, data1, data2, n1, aux)

    
    
    
    #https://en.wikibooks.org/wiki/Introduction_to_Numerical_Methods/Numerical_Differentiation
    #backward divided difference method

   


    estimated_game_parameters = game_parameters
    estimated_simulation_parameters = simulation_parameters



    A2 = @spawnat aux solve_tree(data2, game_parameters)
    A1 = solve_tree(data1, game_parameters)
    A2 = fetch(A2)
    
    
    
    simulated_moments = simulate_moments(S, N, game_parameters, simulation_parameters, data1, data2, A1, A2, n1)

    P = size(game_parameters,1)+size(simulation_parameters,1)
    M = size(simulated_moments,1)
    D = zeros(M, P)
    
    #porcentagem de variação no parâmetro
    pct = 1/100
    
    for p in P:-1:1

        if(p>size(game_parameters,1))

            simulation_parameters = copy(estimated_simulation_parameters)
            
            delta = simulation_parameters[p-size(game_parameters,1)] * pct

            simulation_parameters[p-size(game_parameters,1)] -= delta


            if(p == 5 || p ==6) #para ajustar caso seja habilidade inicial de S ou de J
                simulation_parameters[p-size(game_parameters,1)] = round(simulation_parameters[p-size(game_parameters,1)], digits=0)
            end
            
            
            #caso a variação percentual não seja suficiente para reduzir habilidades iniciais, delta será 1
            if(simulation_parameters[p-size(game_parameters,1)] == estimated_simulation_parameters[p-size(game_parameters,1)])
                delta = Int64(1)
                
                simulation_parameters[p-size(game_parameters,1)] -= delta
            end

            new_simulated_moments = simulate_new_moments(p, S, N, game_parameters, simulation_parameters, data1, data2, A1, A2, n1)

            D[:, p] = (simulated_moments - new_simulated_moments) ./ delta
            
        else

            #se for um parâmetro de jogo, precisa recalcular o jogo

            game_parameters = copy(estimated_game_parameters)
            
            delta = game_parameters[p] * pct
            game_parameters[p] -= delta

            A2 = @spawnat aux solve_tree(data2, game_parameters)
            A1 = solve_tree(data1, game_parameters)
            A2 = fetch(A2)



            new_simulated_moments = simulate_new_moments(p, S, N, game_parameters, simulation_parameters, data1, data2, A1, A2, n1)

            D[:, p] = (simulated_moments - new_simulated_moments) ./ delta


        end


    end
    
    
    
    return D
end
    

derivative_moments (generic function with 1 method)

In [30]:
Derivative = derivative_moments(S, N, game_parameters, simulation_parameters, data1, data2, n1, aux)

 47.812134 seconds (141.59 M allocations: 3.154 GiB, 2.63% gc time)
      From worker 3:	 36.023834 seconds (141.59 M allocations: 3.154 GiB, 1.82% gc time)
parameter 7[0.639, 8.553, 0.01, 0.011][11.0, 90.0, 4.4975700000000005]
parameter 6[0.639, 8.553, 0.01, 0.011][11.0, 89.0, 4.543]
parameter 5[0.639, 8.553, 0.01, 0.011][10.0, 90.0, 4.543]
       From worker 3:	 53.414062 seconds (141.59 M allocations: 3.154 GiB, 18.75% gc time)
71.382463 seconds (141.59 M allocations: 3.154 GiB, 33.65% gc time)
parameter 4[0.639, 8.553, 0.01, 0.010889999999999999][10.0, 90.0, 4.543]
       From worker 3:	 43.159837 seconds (141.59 M allocations: 3.154 GiB, 1.75% gc time)
48.536867 seconds (141.59 M allocations: 3.154 GiB, 1.18% gc time)
parameter 3[0.639, 8.553, 0.0099, 0.011][10.0, 90.0, 4.543]
       From worker 3:	 42.902130 seconds (141.59 M allocations: 3.154 GiB, 1.16% gc time)
49.739593 seconds (141.59 M allocations: 3.154 GiB, 1.44% gc time)
parameter 2[0.639, 8.46747, 0.01, 0.011][10.0, 90.

8×7 Array{Float64,2}:
   1.30515    -0.271256    -1298.28    -1289.38   …  -0.00579645   0.0908207
  -2.29123     0.246008     3795.15     3638.48       0.0114915    0.133209
   1.32089    -0.335136     -924.149    -946.019     -0.012412     0.113216
   0.331958    0.00708594   7351.52     6673.55       0.00521886  -0.00667027
 -14.7515     -0.314492    -3540.79    -3201.93      -0.172943     1.62298
 -11.1003     -0.214215    -1867.52    -1673.07   …  -0.131371     1.79867
  -0.0167323  -0.0942029      65.6934    158.822      0.0013325   -0.065914
   0.0712664   0.0869639   -4248.81    -3818.33       0.0122835    0.088857

In [31]:
Derivative[:,2]

8-element Array{Float64,1}:
 -0.27125629181626487
  0.24600758115314386
 -0.33513560482442567
  0.00708594184567402
 -0.31449220582857806
 -0.21421507792458255
 -0.09420287591189372
  0.08696391795142722

In [32]:
using DelimitedFiles

writedlm( "Derivative-nocd90_1pct.csv",  Derivative, ',')